<img src="./figures/rhdhv_logo.jpg" width=200 height=400 align="right" />
<img src="./figures/vallei_veluwe.png" width=200 height=400 align="right" />

***

# Profile Optimizer DHydro - Jupyter notebook 


****


***

## Introductie
De **Profile Optimizer** is een Python-tool waarin het optimaliseren van profielen voor D-Hydro modellen geautomatiseerd is. Gebaseerd op een bestaand D-Hydro model, zal een deel van het systeem worden gewijzigd om een geoptimaliseerde situatie te vinden. In deze versie (v1.0) is het mogelijk om de bodembreedte van één profiel van één tak te optimaliseren aan de hand van de huidige bodembreedte en de helling. De bodembreedte wordt geoptimaliseerd, zodat een gewenste stroomsnelheid ontstaat bij de gekozen profiel locatie. 

Deze notebook is opgezet tijdens TKI4 als workflow voor de Pilot van de Profile Optimizer bij Waterschap Vallei en Veluwe. 


## Contact 
De Profile Optimizer is onderdeel van HYDROLIB, een open source community voor python tools voor het D-Hydro software pakket. Bezoek de website van Hydrolib voor meer informatie: https://github.com/Deltares/HYDROLIB

De Profile Optimizer is ontwikkeld door Royal HaskoningDHV:
- rineke.hulsman@rhdhv.com
- lisa.weijers@rhdhv.com
- valerie.demetriades@rhdhv.com

****

### Content

* [Stap 0: Klaarzetten input](#stap0)

* [Stap 1: Kies optimalisatie gebied/locatie](#stap1)

* [Stap 2: Optimaliseer de bodembreedte](#stap2)
    * [Stap 2.1: Kies de startwaarde voor de bodembreedte](#stap2.1)
    * [Stap 2.2: Genereer optimalisatie window voor de bodembreedte](#stap2.2)
    * [Stap 2.3: Reken de modellen door voor de bodembreedtes in de optimalisatie window](#stap2.3)
    * [Stap 2.4: Voer het optimalisatie-algoritme uit](#stap2.4)
    
* [Stap 3: Controleer of de bodembreedte voldoet](#stap3)

* [Stap 4: Maak een schets van de geoptimaliseerde profielen](#stap4)

* [Stap 5: Kies of je het model met geoptimaliseerde profiel wilt bewaren](#stap5)


*****
Import general packages:

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from pathlib import Path
import tqdm

***

### Stap 0: Klaarzetten input <a class="anchor" id="stap0"></a>

In deze stap selecteren we het model waarvoor we de Profile Optimizer willen gaan gebruiken. Uit het geselecteerde model zijn een aantal bestanden essentieel voor deze tool. Die worden in deze stap geïdentificeerd.

Tevens worden de werkmappen en outputfolder aangemaakt. 

### Mappen:

`temp_folder`: tijdelijke map waarin iteraties worden weggeschreven. Deze map mag nog niet bestaan en kan aan het einde van het process automatisch verwijderd worden.  
`output_folder`: map waarin het geoptimaliseerde model wordt weggeschreven. Deze map mag nog niet bestaan. 

In [ ]:
temp_folder = 'C:/local/wvv_test'
output_folder = 'C:/local/wvv_temp'

### D-Hydro model input

`model_map`: de map waarin het bron model staat (als `Path` voor makkelijke relatieve paden vanaf hier).    
`model_mdu`: de MDU van dit model (verwijs relatief op deze manier: `model_map/'model_naam.mdu'`).  
`model_network_nc`: naam van het netwerk bestand van dit model.  
`crossdef_filename`: naam van het bestand waarin de cross section definities van dit model staan.  
`crossloc_filename`: naam van het bestand waarin de cross section locaties van dit model staan.  
`bat_file`: naam van de batchfile waarmee de DIMR berekening wordt uitgevoerd. 

In [ ]:
model_map = Path(r'c:\Users\908367\Box\BH6657 TKI4 DHYDRO\BH6657 TKI4 DHYDRO WIP\08_Oppervlaktewater\1_modellen\Project1.dsproj_data\FlowFM\input')
model_mdu = model_map/'FlowFM.mdu'
model_network_nc = model_map/'FlowFM_net.nc'
crossdef_filename = model_map/'crsdef.ini'
crossloc_filename = model_map/'crsloc.ini' 
bat_file = model_map/'run.bat'

### Hydrologische kenmerken en uitgangspunten:

`u_gewenst`: doelwaarde stroomsnelheid. $[m/s]$  
`afvoer`: debiet in de waterloop die geoptimaliseerd wordt. $[m^3/s]$  
`waterdiepte`: gewenste/verwachtte waterdiepte in waterloop die geoptimaliseerd wordt. $[m]$  
`profieldiepte`: diepte van het trapeziumprofiel. Wordt alleen gebruikt in schematisatie. Diepte van profiel wordt gebaseerd op bronmodel, profieldiepte bepaalt tot hoe hoog de oevers lopen. $[m]$  
`talud_profiel`: gewenst talud van het profiel. $[m/m]$  
`verhang`: verhang binnen de waterloop die geoptimaliseerd wordt. $[m/m]$  
`strickler_ks`: ruwheid van de waterloop als strickler ks coefficient. $[\dfrac{m^{1/3}}{s}]$  

In [ ]:
u_gewenst = 0.22   
afvoer = 0.368        
waterdiepte = 0.60
profieldiepte = 1.5
talud_profiel = 2   
verhang = 0.8/1000     
strickler_ks = 10       

### Input voor optimalisatie:

`shapefile_path`: shapefile (polygoon) waarmee het optimalisatie gebied wordt geselecteerd.  
`check_point`: X & Y coordinaten (RD New) waar de berekende stroomsnelheid wordt gecontroleerd. 

In [ ]:
shapefile_path = r'c:\local\grift.gpkg'
check_point = {'x': 194746, 'y': 470795} 

***
### Stap 1: Kies optimalisatie gebied/locatie <a class="anchor" id="stap1"></a>

In deze stap selecteren we een deel van het model waarvoor we de profielen gaan optimaliseren. We gebruiken hiervoor de functie *selected_area_for_optimization* van *select_area.py*. Let op dat in deze versie van de Profile Optimizer je alleen één shapefile kunt opgeven van één bepaalde branch. Je selecteert dan alle profielen binnen deze branch en opgegeven shapefile. Zie het voorbeeld hieronder. 

<img src="./figures/selectie_profielen.png" width=400 />

In [ ]:
from profile_optimizer.geometry import create_branches, create_crosssections, select_crosssection_locations

#Run de functie en print het resultaat
branches = create_branches(model_network_nc)
crosssection_locations = create_crosssections(branches, crossloc_filename)
selected_profiles = select_crosssection_locations(crosssection_locations, shapefile_path)
selected_profiles

***
### Stap 2: Optimaliseer de bodembreedte <a class="anchor" id="stap2"></a>

#### Stap 2.1: Eerste inschatting bodembreedte <a class="anchor" id="stap2.1"></a>

Voor het optimaliseren van de bodembreedte bij een gewenste stroomsnelheid wordt begonnen met een inschatting van de bodembreedte. Deze waarde wordt berekend met de Manning formule voor de gewenste snelheid V bij een gegeven verhang, ruwheid, waterdiepte en gekozen talud (stap 0).  Vervolgens wordt de eerste inschatting gecontroleerd met $Q=V*A$, aangezien de eerste schatting met manning geen rekening houdt met het totale debiet. Bij deze controle (`check_QVA`) wordt de bodembreedte in stappen van 5% nog verder aangepast om te zorgen dat het debiet past bij de breedte. Is het berekende debiet met de gevonden bodembreedte te laag, dan wordt de bodembreedte vergroot tot er voldoende debiet is en vice versa. 

`eerste_schatting_breedte` is een bodembreedte op basis van Manning, voor controle van het debiet.  
`nadere_schatting_breedte` is de aangepaste bodembreedte welke past bij het debiet. Deze wordt gebruikt als eerste inschatting binnen het optimalisatie window in de volgende stap. 

In [ ]:
from profile_optimizer.preprocessing import bottom_width, check_QVA

eerste_schatting_breedte = bottom_width(strickler_ks, verhang, talud_profiel, waterdiepte, u_gewenst)[0]
nadere_schatting_breedte = check_QVA(afvoer, waterdiepte, talud_profiel, eerste_schatting_breedte, verhang, strickler_ks)
print (f"De geschatte bodembreedte op basis van Manning & Q=V*A: {nadere_schatting_breedte:.2f} m")

In [ ]:
nadere_schatting_breedte

#### Stap 2.2: Genereer optimalisatie window voor de bodembreedte <a class="anchor" id="stap2.2"></a>

Voor het optimalisatie algoritme definiëren we een optimalisatie window van de bodembreedte om de 'nadere schatting breedte' van stap 2.1 heen. Je kunt hierbij zelf aangeven wat de bandbreedte (in %) is t.o.v. de gekozen startwaarde en hoeveel waardes je in de window wilt definiëren. Het is tevens mogelijk om in plaats van de `nadere_schatting_breedte` een eigen gekozen breedte te gebruiken, indien je een gericht optimalisatie window zelf wilt instellen. Vul dan zelf een waarde in voor `b_start_value`.

In [ ]:
#Importeer functie
from profile_optimizer.preprocessing import search_window

#Definieer de argumenten voor deze functie
b_start_value = float(nadere_schatting_breedte) 
# b_start_value = 7
bandwidth_perc = 50
iterations = 5

#Run de functie en print het resultaat
window_b = search_window(b_start_value, bandwidth_perc, iterations)

print(f"Het optimalisatie window loopt van {min(window_b):.2f} m tot {max(window_b):.2f} m")
print(f"De volgende bodembreedtes worden binnen het optimalisatie window doorgerekend:")
print([round(b, 2) for b in window_b])

# print ("Voor de volgende bodembreedtes - binnen het optimalisatie window - gaan we het model doorrekenen om de bijbehorende stroomsnelheid te berekenen:      "+str(np.round(window_b,2)))

#### Stap 2.3: Reken de modellen door voor de bodembreedtes in de optimalisatie window <a class="anchor" id="stap2.3"></a>

Op basis van het gerealiseerde optimalisatie window van de bodembreedte b wordt nu voor iedere bodembreedte het model doorgerekend en de resultaten (Q=debiet, V=stroomsnelheid, d=waterdiepte) weggeschreven in een dataframe. Ook wordt de gekozen bodembreedte (B) hierin opgenomen.

In [ ]:
from profile_optimizer.postprocessing import Results
from profile_optimizer.optimizer import ProfileOptimizer

result_list = []

optimizer = ProfileOptimizer(base_model_fn = model_mdu,
                            bat_file = bat_file,
                            work_dir = temp_folder,
                            output_dir = output_folder,
                            iteration_name = 'Demo')

for bottom_width in tqdm.tqdm(window_b, total=len(window_b)):
    prof = {'depth': profieldiepte, 'bottom_width': bottom_width, 'slope_l': 2, 'slope_r': 2}
    latest_folder = optimizer.create_iteration(list(selected_profiles['definitionid']), prof)
    optimizer.run_latest()
    dfm_output_folder = Path(latest_folder).parent/f"output_{Path(latest_folder).name}"
    result = Results(dfm_output_folder)
    xy_result = result.result_at_xy(check_point['x'], check_point['y'])
    result_list.append(xy_result)

results = pd.concat(result_list, axis=0)
results['B'] = window_b

In [ ]:
results.to_csv(Path(output_folder)/'results.csv')

In [ ]:
results

#### Stap 2.4: Voer het optimalisatie-algoritme uit <a class="anchor" id="stap2.4"></a>

In deze wordt het optimalisatie-algoritme uitgevoerd. Hierbij wordt voor alle doorgerekende bodembreedtes de relatie tussen de bodembreedte (x-as) en de bijbehorende berekende stroomsnelheid (y-as) geplot. Vervolgens wordt het punt dat boven de gewenste stroomsnelheid ligt gekozen en het punt dat eronder ligt. Tussen deze twee punten wordt een lineaire interpolatie uitgevoerd en zo kan de bodembreedte worden berekend die bij de gewenste stroomsnelheid hoort.

* Dit optimalisatie algoritme werkt alleen als V_target (de gewenste stroomsnelheid) tussen twee punten zit van bovenstaande dataframe resultaten. 

In [ ]:
from profile_optimizer.optimizer import find_optimum

window_b = results['B']
u_bij_berekende_window_b = results['V']
berekende_waterdiepte = results['WL']

df, geoptimaliseerde_b = find_optimum(window_b, u_bij_berekende_window_b, u_gewenst, berekende_waterdiepte)
print (f'De geoptimaliseerde bodembreedte is: {geoptimaliseerde_b:.2f} m')

************
### Stap 3: Modelberekening geoptimaliseerde bodembreedte <a class="anchor" id="stap3"></a>

Nu is de geoptimaliseerde bodembreedte verkregen, die volgens het optimalisatie-algoritme bij de gewenste stroomsnelheid hoort. Het model wordt nu nogmaals doorgerekend met de geoptimaliseerde bodembreedte. De gebruiker kan zo het effect op het systeem beoordelen.

In [ ]:
geoptimaliseerde_b 
profiel = {'depth': waterdiepte, 'bottom_width': geoptimaliseerde_b, 'slope_l': talud_profiel, 'slope_r': talud_profiel}
geoptimaliseerde_run = optimizer.create_iteration(list(selected_profiles['definitionid']), profiel)
optimizer.run_latest()

dfm_output_folder = Path(geoptimaliseerde_run).parent/f"DFM_OUTPUT_{Path(geoptimaliseerde_run).name}"
result = Results(dfm_output_folder)
xy_result = result.result_at_xy(check_point['x'], check_point['y'])
xy_result['B'] = geoptimaliseerde_b

Voeg de resultaten van de modelrun van de geoptimaliseerde bodembreedte toe aan de dataframe van resultaten --> als laatste regel.

In [ ]:
results = results.append(xy_result[['geometry', 'Q', 'V', 'WL', 'WD', 'B']])
results

In [ ]:
results.to_csv(Path(output_folder)/'results_with_optimized.csv')

****
### Stap 4: Maak een schets van de geoptimaliseerde profielen: <a class="anchor" id="stap4"></a>

In deze stap kan je een profile_ id opgeven van één locatie waarbij het profiel van het huidige model en van het geoptimaliseerde model weergegeven wordt. Let op dat deze profielen genormaliseerd zijn weergegeven.

In [ ]:
from profile_optimizer.postprocessing import plot_profiles

#Kies een profiel uit je model. Of gebruik het eerste profiel uit het interessegebied. 
profile_id = 'prof_24102013-DP14'
if profile_id is None:
    profile_id = list(selected_profiles['defenitionid'])[0]

#Run de functie en print het resultaat
plot_profiles(model_mdu, talud_profiel, geoptimaliseerde_b, profieldiepte, profile_id)

Wegschrijven alle profielen in de folder van de geoptimaliseerde model run resultaten:

In [ ]:
for i in range(len(selected_profiles['definitionid'])):
    profile_id = selected_profiles['definitionid'].iloc[i]
    profiel_figuur = plot_profiles(model_mdu, talud_profiel, geoptimaliseerde_b, profieldiepte, profile_id)
    profiel_figuur.write_html(str(dfm_output_folder/f'{profile_id}.html'))

***

****
### Stap 5: Kies of je het model met geoptimaliseerde profiel wilt bewaren<a class="anchor" id="stap5"></a>

In deze stap kun je ervoor kiezen om de werkmap met de iteraties weg te gooien en het geoptimaliseerde model over te zetten naar de output folder. Je kunt er ook voor kiezen om een andere iteratie dan de geoptimaliseerde run te bewaren en de rest weg te gooien. 

In [ ]:
optimizer.export_model(specific_iteration="latest", #Can also be an interger of a specific run, for example: 3
                      cleanup=True)